In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['figure.dpi'] = 250
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/dhanesh-sakre/DATA-SCIENCE/refs/heads/main/Assignment/Multiple_Linear_Regression/ToyotaCorolla%20-%20MLR.csv')
df

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

# EDA

In [ ]:
df.describe()

In [ ]:
# descriptive_analysisi:
    # there is no missing row
    # outliers may be present in the KM,cc columns

In [ ]:
sns.boxplot(df)

In [ ]:
# it's confirm that 'KM' and 'cc' column is having outliers

In [ ]:
# check for the missing values

In [ ]:
df.isna().sum()

In [ ]:
# there is no missing values present in the dataset

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
df['Fuel_Type'].unique()

In [ ]:
df.nunique()

In [ ]:
# frequency of unique values in the columns

In [ ]:
df['Doors'].value_counts()

In [ ]:
df['Gears'].value_counts()

In [ ]:
df['Cylinders'].value_counts()

In [ ]:
df['Automatic'].value_counts()

# data transformation

In [ ]:
df.head(3)

### one hot encoding

In [ ]:
# Fuel_Type column is having text data type so I need to perform one_hot_edcoding

In [ ]:
df = pd.get_dummies(df, columns=['Fuel_Type'], dtype=int)
df

# Feature scaling

In [ ]:
x = df.iloc[:, 1:]
x

In [ ]:
y = df[['Price']]
y

In [ ]:
# standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
st = StandardScaler()

In [ ]:
st_x = st.fit_transform(x)
st_x

In [ ]:
x = pd.DataFrame(st_x, columns=x.columns)    #converting to dataframe
x

## model builiding

In [ ]:
# splitting in training and testing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=0.80, random_state=42)

In [ ]:
xtrain

In [ ]:
xtest

In [ ]:
ytrain

In [ ]:
ytest

In [ ]:
# concatinating xtrain and ytrain(target column) data to get a complete training dataset

In [ ]:
train_data = pd.concat([xtrain, ytrain], axis=1)
train_data

In [ ]:
# model building

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
model = smf.ols('Price~Age_08_04+KM+HP+Automatic+cc+Doors+Cylinders+Gears+Weight+Fuel_Type_CNG+Fuel_Type_Diesel+Fuel_Type_Petrol', data=train_data).fit()

In [ ]:
model.params            #intercept and slop value of each independent variable

In [ ]:
# checking multicolinearity

In [ ]:
model.pvalues

### conclusion
 these are the pvalues for each column and all the pvalues are less than alpha value(0.5)
 that means there is no multicolinearity between the independent features
 and all of the independent features are important for analysis because their slow value is not 0.

In [ ]:
model.rsquared                  #checking accuracy

In [ ]:
print(f'accuracy of the model is:  {model.rsquared}')

### checking assumptions about the error datapoint

In [ ]:
import statsmodels.api as sm

In [ ]:
model.fittedvalues               #getting the predicted values that model calculated in backend

In [ ]:
errors = model.resid           #error calculated by the model
errors

# CHECK ASSUMPTIONS ABOUT ERROR DATAPOINTS

## 1st assumption for errordatapoints:
### IID Normal distribution

In [ ]:
# QQPLOT for check normal distribution of error datapoints

In [ ]:
sm.qqplot(errors, line='q')

In [ ]:
# there are 2 outliers present in the lower side

In [ ]:
# row number of error datapoints and error

In [ ]:
df_error = pd.DataFrame(errors,columns=['error'])
df_error = df_error.sort_values(by='error')
df_error

In [ ]:
# row 960 and 601 are outlier datapoints

# 2rd assumption for error datapoints:
###  HOMOSCEDASTICITY

In [ ]:
plt.scatter(model.fittedvalues, errors)
plt.xlabel('predicted values')
plt.ylabel('errors')
plt.title('constant variance')

# 3rd assumption for error datapoints:
### MEAN = 0

In [ ]:
sm.graphics.plot_regress_exog(model, 'Age_08_04')

In [ ]:
sm.graphics.plot_regress_exog(model, 'KM')

In [ ]:
sm.graphics.plot_regress_exog(model, 'HP')

In [ ]:
sm.graphics.plot_regress_exog(model, 'Automatic')

In [ ]:
sm.graphics.plot_regress_exog(model, 'cc')

In [ ]:
sm.graphics.plot_regress_exog(model, 'Doors')

In [ ]:
sm.graphics.plot_regress_exog(model, 'Cylinders')

In [ ]:
sm.graphics.plot_regress_exog(model, 'Gears')

In [ ]:
sm.graphics.plot_regress_exog(model, 'Weight')

### conclusion(interpretation) of testing of assumptions of error datapoints by these 3 types of plots(qqplot, scatterplot, plot_regress_exog)

#### 1.     there is not homoscedasticity due to presence of outlier datapoints

#### 2.     error data points are not following IID distribution because of 2 outlier datapoints

#### 3.     third assumption is getting satisfied:  mean = 0

# GETTING PREDICTION on testing data

In [ ]:
xtest

In [ ]:
model.predict(xtest)       #getting predictions on testing data from model

In [ ]:
ypred = pd.DataFrame(model.predict(xtest), columns=['Predicted_Price'])
ypred

In [ ]:
# using sklearn liberary for calculating accuracy of the model on testing data

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
accuracy = r2_score(ytest, ypred)
accuracy

In [ ]:
print(f'on training data accuracy of model is :  {accuracy}')

In [ ]:
# for my own understanding

In [ ]:
final_observation = pd.concat([xtest, ypred], axis=1)
final_observation

# drop outliers

In [ ]:
df_error

In [ ]:
train_data.drop(960, axis=0, inplace=True)
train_data.drop(601, axis=0, inplace=True)

train_data

## model 2

In [ ]:
model1 = smf.ols('Price~Age_08_04+KM+HP+Automatic+cc+Doors+Cylinders+Gears+Weight+Fuel_Type_CNG+Fuel_Type_Diesel+Fuel_Type_Petrol', data=train_data).fit()

In [ ]:
model1.rsquared  

In [ ]:
# accuracy increased by more than 1 precentage: from 0.8702 to 0.8875 

In [ ]:
model1.predict(xtest)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
accuracy1 = r2_score(ytest, ypred)
accuracy1

### MODEL3

In [ ]:
# if I train regression model on testing data to check the accuracy of model then

In [ ]:
test_data = pd.concat([xtest, ytest], axis=1)
test_data

In [ ]:
model2 = smf.ols('Price~Age_08_04+KM+HP+Automatic+cc+Doors+Cylinders+Gears+Weight+Fuel_Type_CNG+Fuel_Type_Diesel+Fuel_Type_Petrol', data=test_data).fit()

In [ ]:
model2.rsquared

In [ ]:
print(f'accuracy of this model is:   {model2.rsquared}')